In [2]:
import pickle
from keras.models import load_model
import numpy as np
import cv2
# Load calibration data
with open('./pickel_files/calibration_data.pkl', 'rb') as f:
    X, Y = pickle.load(f)


# Initialize your gaze detection model
model = load_model('./models/eye_gaze_v15_sigmoidv8.h5')

# Predict gaze points for each image in the calibration dataset
predicted_gaze_points = []
for image in X:
    image_reshaped = np.expand_dims(image, axis=0)
    predicted_point = model.predict(image_reshaped)  # Assuming model.predict(image) returns the gaze point
    predicted_gaze_points.append(predicted_point)

# Create the dataset for the adjustment model
adjustment_dataset = {
    'predicted_gaze_points': predicted_gaze_points,
    'actual_gaze_points': Y# Assuming 'X' holds the actual gaze points
}

# Save the dataset into a new pickle file
with open('adjustment_dataset.pkl', 'wb') as f:
    pickle.dump(adjustment_dataset, f)


1/1 [==============================] - 0s 26ms/step


In [3]:
with open('./adjustment_dataset.pkl', 'rb') as f:
    data = pickle.load(f)
x= data['predicted_gaze_points']
x = np.squeeze(np.array(x))
x.shape

(756, 2)

In [4]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Load your data
with open('./adjustment_dataset.pkl', 'rb') as f:
    calibration_data = pickle.load(f)


X = np.squeeze(np.array(calibration_data['predicted_gaze_points']), axis=1)  # Remove extra dimensions if necessary
y = np.array(calibration_data['actual_gaze_points'])[:, :2]  # Remove extra dimensions if necessary
# Assuming `calibration_data` is a dictionary with 'predicted_points' and 'actual_points'
# X = calibration_data['predicted_gaze_points']  # Predicted gaze points from the main model
# y = calibration_data['actual_gaze_points']  # Actual gaze points

print(X.shape)
print(y.shape)

# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=None)


# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=None)
model.fit(X_train, y_train)

# Predict on the validation set
y_pred = model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {mse}')



(756, 2)
(756, 2)
Mean Squared Error: 0.024214671971060853


In [5]:
# Save the model
with open('adjustment_model.pkl', 'wb') as f:
    pickle.dump(model, f)